In [ ]:

notebook_path = %pwd
import sys
sys.path.append(notebook_path)


In [ ]:

import os
import glob

import pickle
import pprint
import gym
import copy
import pickle
import warnings

import numpy as np
import pandas as pd
import itertools as it
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from pprint import pprint
from datetime import datetime

from sklearn.mixture import GaussianMixture, BayesianGaussianMixture

from puddle_world.envs import *
from explicit_env.soln import value_iteration, q_from_v, OptimalPolicy, policy_evaluation
from unimodal_irl.utils import empirical_feature_expectations

from multimodal_irl import bv_em_maxent, mixture_ll_maxent, responsibilty_matrix_maxent
from multimodal_irl.metrics import *

from unimodal_irl import sw_maxent_irl
from unimodal_irl.utils import pad_terminal_mdp
from unimodal_irl.metrics import ile_evd


In [ ]:

# Load environments
env_det_wet = CanonicalPuddleWorldEnv(mode='wet', wind=0.0)
env_det_dry = CanonicalPuddleWorldEnv(mode='dry', wind=0.0)
env_det_any = CanonicalPuddleWorldEnv(mode='any', wind=0.0)
_envs_det = [env_det_wet, env_det_dry, env_det_any]
env_stoch_wet = CanonicalPuddleWorldEnv(mode='wet', wind=0.2)
env_stoch_dry = CanonicalPuddleWorldEnv(mode='dry', wind=0.2)
env_stoch_any = CanonicalPuddleWorldEnv(mode='any', wind=0.2)
_envs_stoch = [env_stoch_wet, env_stoch_dry, env_stoch_any]

# Load rollout data
with open("pw-deterministic-wet.pkl", "rb") as file:
    rollouts_det_wet = pickle.load(file)
with open("pw-deterministic-dry.pkl", "rb") as file:
    rollouts_det_dry = pickle.load(file)
with open("pw-deterministic-any.pkl", "rb") as file:
    rollouts_det_any = pickle.load(file)
with open("pw-stochastic-wet.pkl", "rb") as file:
    rollouts_stoch_wet = pickle.load(file)
with open("pw-stochastic-dry.pkl", "rb") as file:
    rollouts_stoch_dry = pickle.load(file)
with open("pw-stochastic-any.pkl", "rb") as file:
    rollouts_stoch_any = pickle.load(file)

# Pre-compute optimal SD policy value for each environment
pi_det_wet_v = policy_evaluation(
    env_det_wet,
    OptimalPolicy(q_from_v(value_iteration(env_det_wet), env_det_wet), stochastic=False)
)
pi_det_dry_v = policy_evaluation(
    env_det_dry,
    OptimalPolicy(q_from_v(value_iteration(env_det_dry), env_det_dry), stochastic=False)
)
pi_det_any_v = policy_evaluation(
    env_det_any,
    OptimalPolicy(q_from_v(value_iteration(env_det_any), env_det_any), stochastic=False)
)
_pi_vs_det = [pi_det_wet_v, pi_det_dry_v, pi_det_any_v]

pi_stoch_wet_v = policy_evaluation(
    env_stoch_wet,
    OptimalPolicy(q_from_v(value_iteration(env_stoch_wet), env_stoch_wet), stochastic=False)
)
pi_stoch_dry_v = policy_evaluation(
    env_stoch_dry,
    OptimalPolicy(q_from_v(value_iteration(env_stoch_dry), env_stoch_dry), stochastic=False)
)
pi_stoch_any_v = policy_evaluation(
    env_stoch_any,
    OptimalPolicy(q_from_v(value_iteration(env_stoch_any), env_stoch_any), stochastic=False)
)
_pi_vs_stoch = [pi_stoch_wet_v, pi_stoch_dry_v, pi_stoch_any_v]


In [ ]:

# Load the dataset
filename = "CanonicalPuddleWorld-stochastic-2mode-experiments.csv"
df = pd.read_csv(filename)
_envs = _envs_stoch
_env = _envs[0]
_pi_vs = _pi_vs_stoch

# Remove invalid index column
del df["Unnamed: 0"]

# Add columns for evaluation metrics
df["Negative Log Likelihood"] = np.nan
df["Normalized Information Distance"] = np.nan
df["Adjusted Normalized Information Distance"] = np.nan
df["Min Cost Flow ILE"] = np.nan
df["Min Cost Flow EVD"] = np.nan
df["Mean ILE"] = np.nan
df["Mean EVD"] = np.nan

df


In [ ]:


for _exp in tqdm(range(len(df))):
    exp = df.iloc[_exp]
#     print("Evaluating... {}/{} ({}-{} {}-{}) ".format(
#         _exp,
#         len(df),
#         exp["Transition Type"],
#         int(exp["Num GT Clusters"]),
#         exp["Initialisation"],
#         int(exp["Num Learned Clusters"])
#     ), end="")
    
    rollouts_per_mode = exp["Num Rollouts"] // exp["Num GT Clusters"]
    # Get the exact set of rollouts used for this experiment
    start_idx = int(exp["Replicate"] * rollouts_per_mode)
    end_idx = int((exp["Replicate"] + 1) * rollouts_per_mode)
    
    if exp["Transition Type"] == 'deterministic':
        if exp["Num GT Clusters"] == 2:
            rollouts = [
                *rollouts_det_wet[start_idx:end_idx],
                *rollouts_det_dry[start_idx:end_idx]
            ]
            gt_rewards = [
                env_det_wet.state_rewards,
                env_det_dry.state_rewards
            ]
        else:
            rollouts = [
                *rollouts_det_wet[start_idx:end_idx],
                *rollouts_det_dry[start_idx:end_idx],
                *rollouts_det_any[start_idx:end_idx]
            ]
            gt_rewards = [
                env_det_wet.state_rewards,
                env_det_dry.state_rewards,
                env_det_any.state_rewards
            ]
    else:
        if exp["Num GT Clusters"] == 2:
            rollouts = [
                *rollouts_stoch_wet[start_idx:end_idx],
                *rollouts_stoch_dry[start_idx:end_idx]
            ]
            gt_rewards = [
                env_stoch_wet.state_rewards,
                env_stoch_dry.state_rewards
            ]
        else:
            rollouts = [
                *rollouts_stoch_wet[start_idx:end_idx],
                *rollouts_stoch_dry[start_idx:end_idx],
                *rollouts_stoch_any[start_idx:end_idx]
            ]
            gt_rewards = [
                env_stoch_wet.state_rewards,
                env_stoch_dry.state_rewards,
                env_stoch_any.state_rewards
            ]
    
    # Compute GT responsibility matrix
    resp_gt = responsibilty_matrix_maxent(
        _env,
        rollouts,
        int(exp["Num GT Clusters"]),
        state_reward_weights=gt_rewards
    )
    
    # Get model responsibility matrix
    resp_learned = np.array(eval(exp["Responsibility Matrix"]))
    mode_weights_learned = np.sum(resp_learned, axis=0) / resp_learned.shape[0]
    
    # Get model reward weights
    learned_rewards = np.array(eval(exp["Reward Weights"]))
    
    # Compute the log-likelihood of this model
    df.at[_exp, "Negative Log Likelihood"] = -1.0 * mixture_ll_maxent(
        _env,
        rollouts,
        mode_weights_learned,
        learned_rewards
    )
    
    # Compute the aNID of this model
    df.at[_exp, "Normalized Information Distance"] = normalized_information_distance(resp_learned, resp_gt)
    df.at[_exp, "Adjusted Normalized Information Distance"] = adjusted_normalized_information_distance(resp_learned, resp_gt)
    
    # Build error matrices
    ile_mat = np.zeros((resp_learned.shape[1], resp_gt.shape[1]))
    evd_mat = np.zeros((resp_learned.shape[1], resp_gt.shape[1]))
    for learned_idx in range(resp_learned.shape[1]):
        _env_learned = copy.deepcopy(_env)
        _env_learned._state_rewards = learned_rewards[learned_idx]
        for gt_idx in range(resp_gt.shape[1]):
            _env_gt = _envs[gt_idx]
            _pi_vs_gt = _pi_vs[gt_idx]
            
            ile_mat[learned_idx, gt_idx], evd_mat[learned_idx, gt_idx] = ile_evd(
                _env_gt,
                _env_learned,
                optimal_policy_value=_pi_vs_gt
            )
    
    df.at[_exp, "Min Cost Flow ILE"], _ = min_cost_flow_error_metric(resp_learned, resp_gt, ile_mat)
    df.at[_exp, "Min Cost Flow EVD"], _ = min_cost_flow_error_metric(resp_learned, resp_gt, evd_mat)
    
    df.at[_exp, "Mean ILE"] = mean_error_metric(resp_learned, resp_gt, ile_mat)
    df.at[_exp, "Mean EVD"] = mean_error_metric(resp_learned, resp_gt, evd_mat)


In [ ]:

filename_out = filename.replace(".csv", "-metrics.csv")
df.to_csv(filename_out, index=False)
